In [ ]:
# from statsmodels.tsa.arima.model import ARIMA
# gold = [1,1,1,2,1,2,2,2,2,2,1,2,3,3,3,3,2,3,3,4,3,4]
# model = ARIMA(endog=gold, order=(2,1,2))
# res = model.fit()
# print(res.forecast(20))

In [ ]:
from pandas import read_csv, concat, DataFrame
from numpy import nan
from tqdm import trange

In [ ]:
gold_csv = read_csv("./2022c/data/LBMA-GOLD.csv")
bit_csv = read_csv("./2022c/data/BCHAIN-MKPRU.csv")

In [ ]:
time_df = bit_csv["Date"]
time_df.head()
time_array = time_df.to_list()  # 时间ndarray


In [ ]:
# 对比需要插值的地方
print(time_array[:15], "\n", gold_csv["Date"].to_list()[:15])
# 需要插值的位置: 0, 6, 13 --> 不行, 不一定是双修

In [ ]:
# 获取黄金价格, 并插值
days_need_to_fill_nan = set(time_array) - set(gold_csv["Date"].tolist())
# days_need_to_fill_from_csv = set(gold_csv["Date"].tolist())
len(days_need_to_fill_nan)

In [ ]:
gold_df_with_nan = bit_csv.copy()
for i,j in enumerate(time_array):
    if j in days_need_to_fill_nan:
        idx = gold_df_with_nan[gold_df_with_nan["Date"]==j].index.values[0]
        gold_df_with_nan["Value"][idx] = nan
    else:
        gold_df_with_nan["Value"][i] = gold_csv[gold_csv["Date"]==j]["USD (PM)"]
    
gold_df_with_nan.head(20)

In [ ]:
gold_df_with_nan["Value"].interpolate(inplace=True)

In [ ]:
gold_df_with_nan.loc[0]['Value'] = 1300
gold_df_with_nan.head(20)
gold_df_preprocessed = gold_df_with_nan.copy()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(gold_df_preprocessed['Value'])

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
max_time_step = len(gold_df_preprocessed['Value'].to_list())
gold_price = gold_df_preprocessed['Value'].tolist()
start = 1000
window_size = 5
history = gold_price[:start]
predictions = []
for i in trange(start,len(gold_price)):
    try:
        history.append(gold_price[i])
        model = ARIMA(endog=history[-window_size:], order=(2,1,2)).fit()
        output = model.forecast()[0]
        predictions.append(output+50)
    except:
        print("LU error!")
        continue
plt.plot(gold_price, color='blue')
plt.plot(gold_price[:start] + predictions, color='red')
plt.show()

In [ ]:
model.summary()

In [ ]:
max_time_step